# HISTORY

#### Context
* The Laser Interferometer Gravitational-Wave Observatory (LIGO) was designed to open the field of gravitational-wave astrophysics through the direct detection of gravitational waves predicted by Einstein’s General Theory of Relativity. LIGO’s multi-kilometer-scale gravitational wave detectors use laser interferometry to measure the minute ripples in space-time caused by passing gravitational waves from cataclysmic cosmic events such as colliding neutron stars or black holes, or by supernovae. LIGO consists of two widely-separated interferometers within the United States—one in Hanford, Washington and the other in Livingston, Louisiana—operated in unison to detect gravitational waves.

* This dataset is made up of LIGO images that have been classified by people as part of the Gravity Spy Zooniverse project and they've been placed into train,validation and test folders,ready for machine learning.

# PACKAGES AND LIBRARIES

In [ ]:
#GENERAL
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
#PATH PROCESS
import os
import os.path
from pathlib import Path
import glob
#IMAGE PROCESS
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.applications.vgg16 import preprocess_input, decode_predictions
#SCALER & TRANSFORMATION
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from sklearn.preprocessing import LabelEncoder
#ACCURACY CONTROL
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
#OPTIMIZER
from keras.optimizers import RMSprop,Adam,Optimizer,Optimizer, SGD
#MODEL LAYERS
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN,\
LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Convolution2D, ZeroPadding2D,Reshape, Conv2DTranspose, LeakyReLU, GaussianNoise
from keras import models
from keras import layers
import tensorflow as tf
from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K
from keras.utils import plot_model
from keras.datasets import mnist
import keras
#SKLEARN CLASSIFIER
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
#IGNORING WARNINGS
from warnings import filterwarnings
filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning) 
filterwarnings("ignore", category=UserWarning)

# PATH & LABEL PROCESS

In [ ]:
Wave_Train_Data = Path("../input/gravity-spy-gravitational-waves/train/train")
Wave_Test_Data = Path("../input/gravity-spy-gravitational-waves/test/test")
Wave_Validation_Data = Path("../input/gravity-spy-gravitational-waves/validation/validation")

In [ ]:
Train_PNG_Path = list(Wave_Train_Data.glob(r"*/*.png"))
Test_PNG_Path = list(Wave_Test_Data.glob(r"*/*.png"))
Validation_PNG_Path = list(Wave_Validation_Data.glob(r"*/*.png"))

In [ ]:
Train_PNG_Labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],Train_PNG_Path))
Test_PNG_Labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],Test_PNG_Path))
Validation_PNG_Labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],Validation_PNG_Path))

In [ ]:
Train_PNG_Path_Series = pd.Series(Train_PNG_Path,name="PNG").astype(str)
Test_PNG_Path_Series = pd.Series(Test_PNG_Path,name="PNG").astype(str)
Validation_PNG_Path_Series = pd.Series(Validation_PNG_Path,name="PNG").astype(str)

In [ ]:
Train_PNG_Labels_Series = pd.Series(Train_PNG_Labels,name="CATEGORY")
Test_PNG_Labels_Series = pd.Series(Test_PNG_Labels,name="CATEGORY")
Validation_PNG_Labels_Series = pd.Series(Validation_PNG_Labels,name="CATEGORY")

In [ ]:
Main_Train_Data = pd.concat([Train_PNG_Path_Series,Train_PNG_Labels_Series],axis=1)
Main_Test_Data = pd.concat([Test_PNG_Path_Series,Test_PNG_Labels_Series],axis=1)
Main_Validation_Data = pd.concat([Validation_PNG_Path_Series,Validation_PNG_Labels_Series],axis=1)

In [ ]:
Main_Train_Data = Main_Train_Data.sample(frac=1).reset_index(drop=True)
Main_Test_Data = Main_Test_Data.sample(frac=1).reset_index(drop=True)
Main_Validation_Data = Main_Validation_Data.sample(frac=1).reset_index(drop=True)

# IMAGE GENERATOR

In [ ]:
Train_Generator = ImageDataGenerator(rescale=1./255,
                              zoom_range=0.3,
                              shear_range=0.3,
                              horizontal_flip=True,
                              vertical_flip=True,
                              fill_mode="nearest",
                               height_shift_range=0.2,
                              width_shift_range=0.2)

In [ ]:
Val_Test_Generator = ImageDataGenerator(rescale=1./255)

#### HOW TO LOOK WITH GENERATOR

In [ ]:
example_IMG = Main_Train_Data["PNG"][13945]
load_img = image.load_img(example_IMG,target_size=(180,180))
Array_Image = image.img_to_array(load_img)
Array_Image = Array_Image.reshape((1,) + Array_Image.shape)

i = 0 
for batch in Train_Generator.flow(Array_Image,batch_size=1):
    plt.figure(i)
    IMG = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break
plt.show()

#### SPLITTING TRAIN, VALIDATION, TEST

In [ ]:
Train_IMG_Set = Train_Generator.flow_from_dataframe(dataframe=Main_Train_Data,
                                                   x_col="PNG",
                                                   y_col="CATEGORY",
                                                   color_mode="rgb",
                                                   class_mode="categorical",
                                                   subset="training",
                                                    batch_size=32,
                                                   target_size=(180,180))

In [ ]:
Validation_IMG_Set = Val_Test_Generator.flow_from_dataframe(dataframe=Main_Validation_Data,
                                                        x_col="PNG",
                                                        y_col="CATEGORY",
                                                        color_mode="rgb",
                                                        class_mode="categorical",
                                                           batch_size=32,
                                                           target_size=(180,180))

In [ ]:
Test_IMG_Set = Val_Test_Generator.flow_from_dataframe(dataframe=Main_Test_Data,
                                                        x_col="PNG",
                                                        y_col="CATEGORY",
                                                        color_mode="rgb",
                                                        class_mode="categorical",
                                                           batch_size=32,
                                                           target_size=(180,180),
                                                     shuffle=False)

In [ ]:
for data_batch,label_batch in Train_IMG_Set:
    print("DATA SHAPE: ",data_batch.shape)
    print("LABEL SHAPE: ",label_batch.shape)
    break

In [ ]:
for data_batch,label_batch in Validation_IMG_Set:
    print("DATA SHAPE: ",data_batch.shape)
    print("LABEL SHAPE: ",label_batch.shape)
    break

In [ ]:
for data_batch,label_batch in Test_IMG_Set:
    print("DATA SHAPE: ",data_batch.shape)
    print("LABEL SHAPE: ",label_batch.shape)
    break

In [ ]:
print("TRAIN: ")
print(Train_IMG_Set.class_indices)
print(Train_IMG_Set.classes[0:5])
print(Train_IMG_Set.image_shape)
print("---"*20)
print("VALIDATION: ")
print(Validation_IMG_Set.class_indices)
print(Validation_IMG_Set.classes[0:5])
print(Validation_IMG_Set.image_shape)
print("---"*20)
print("TEST: ")
print(Test_IMG_Set.class_indices)
print(Test_IMG_Set.classes[0:5])
print(Test_IMG_Set.image_shape)

# CNN

In [ ]:
Model = Sequential()

Model.add(Conv2D(24,(3,3),activation="relu",input_shape=(180,180,3)))
Model.add(BatchNormalization())
Model.add(MaxPooling2D((2,2),strides=2))

Model.add(Conv2D(64,(3,3),activation="relu",padding="same"))
Model.add(Dropout(0.3))
Model.add(MaxPooling2D((2,2),strides=2))

Model.add(Conv2D(64,(3,3),activation="relu",padding="same"))
Model.add(Dropout(0.3))
Model.add(MaxPooling2D((2,2),strides=2))

Model.add(Conv2D(128,(3,3),activation="relu",padding="same"))
Model.add(Conv2D(128,(3,3),activation="relu",padding="same"))
Model.add(Dropout(0.3))
Model.add(MaxPooling2D((2,2),strides=2))

Model.add(Conv2D(256,(3,3),activation="relu",padding="same"))
Model.add(Dropout(0.3))
Model.add(MaxPooling2D((2,2),strides=2))

Model.add(Flatten())
Model.add(Dense(2352,activation="relu"))
Model.add(Dropout(0.5))
Model.add(Dense(22,activation="softmax"))

In [ ]:
Call_Back = tf.keras.callbacks.EarlyStopping(monitor="loss",patience=3,mode="min")

In [ ]:
Model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
CNN_Model = Model.fit(Train_IMG_Set,validation_data=Validation_IMG_Set,callbacks=Call_Back,epochs=50)

In [ ]:
Model_Results = Model.evaluate(Test_IMG_Set,verbose=False)
print("LOSS:  " + "%.4f" % Model_Results[0])
print("ACCURACY:  " + "%.2f" % Model_Results[1])

In [ ]:
plt.plot(CNN_Model.history["accuracy"])
plt.plot(CNN_Model.history["val_accuracy"])
plt.ylabel("ACCURACY")
plt.legend()
plt.show()

In [ ]:
plt.plot(CNN_Model.history["loss"])
plt.plot(CNN_Model.history["val_loss"])
plt.ylabel("LOSS")
plt.legend()
plt.show()

In [ ]:
plt.plot(CNN_Model.history["loss"])
plt.plot(CNN_Model.history["accuracy"])
plt.ylabel("LOSS - ACCURACY")
plt.legend()
plt.show()

In [ ]:
Dict_Summary = pd.DataFrame(CNN_Model.history)
Dict_Summary.plot()